<a href="https://colab.research.google.com/github/JayroMartinez/game_of_life_sim/blob/main/game_of_life_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
if [ ! -d game_of_life_sim ]; then
  git clone https://github.com/JayroMartinez/game_of_life_sim.git -q
else
  git -C game_of_life_sim pull -q
fi
pip install -q -r game_of_life_sim/requirements.txt

import os, sys
sys.path.append("/content/game_of_life_sim/src")
os.chdir("/content/game_of_life_sim")
print("Environment ready.")

In [ ]:
%run -i run_sim.py --size 60 --prob 0.25

In [ ]:
!python run_sim.py --runs 10000 --size 60 --prob 0.25

In [ ]:
%%bash
cat >> ~/.ssh/config << 'EOF'
Host github.com
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_ed25519_life
  StrictHostKeyChecking no
EOF

In [ ]:
%%bash
# Leer tu autoría desde Drive
AUTHOR_LINE="$(cat /content/drive/MyDrive/keys/author.txt)"

# Extraer nombre y correo
NAME="$(echo "$AUTHOR_LINE" | sed -E 's/^(.*) <.*>$/\1/')"
EMAIL="$(echo "$AUTHOR_LINE" | sed -E 's/^.*<(.+)>$/\1/')"

REPO_DIR="/content/game_of_life_sim"
REMOTE="git@github.com:JayroMartinez/game_of_life_sim.git"

# Añadir el CSV
git -C "$REPO_DIR" add data/results.csv

# Commit con author + committer configurados inline
git -C "$REPO_DIR" \
    -c user.name="$NAME" \
    -c user.email="$EMAIL" \
    commit --author="$AUTHOR_LINE" -m "Add new results"

# Push por SSH (usa tu deploy key)
git -C "$REPO_DIR" push "$REMOTE" HEAD:main